In [1]:
%run Package_AND_Parameter.ipynb
%run Feature_Selection_Methods.ipynb

C:\Users\vita7\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
tr_df = pd.read_csv('tr_df.csv', header=[0,1], index_col=[0,1])
tr_df.index.names = ['epoch','condition']
tr_df.reset_index(level = 1,inplace = True)

train_X = tr_df.drop(columns=['condition'])
train_Y = tr_df['condition']

C:\Users\vita7\Anaconda3\lib\site-packages\pandas\core\generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [3]:
tr_df.head()

Function      condition Spect_slope                                    \
Band                     EEG Fpz-Cz EEG Fpz-Cz Delta EEG Fpz-Cz Theta   
epoch                                                                   
2750      Sleep stage W  -24.945670       -23.984789        -0.226054   
2751      Sleep stage W  -23.769731       -22.905892        -0.215343   
2752      Sleep stage W  -24.182665       -23.310922        -0.224329   
2753      Sleep stage W  -24.692606       -23.712681        -0.232493   
2754      Sleep stage W  -24.218212       -23.309319        -0.238958   

Function                                                                   \
Band     EEG Fpz-Cz Alpha EEG Fpz-Cz Sigma EEG Fpz-Cz Beta EEG Fpz-Cz DOR   
epoch                                                                       
2750             0.237275         0.173856        0.780128       0.961481   
2751             0.237430         0.172868        0.779275       0.963658   
2752             0.237022         0.175659        0.779543       0.963952   
2753             0.227249         0.168636        0.777726       0.960315   
2754             0.236427         0.174723        0.778467       0.962471   

Function                                ...       AutoCorr                 \
Band     EEG Fpz-Cz TOR EEG Fpz-Cz AOR  ... EEG Fpz-Cz TDR EEG Fpz-Cz ADR   
epoch                                   ...                                 
2750           0.009062      -0.009512  ...       0.949501       0.824361   
2751           0.009060      -0.009989  ...       0.957307       0.821846   
2752           0.009276      -0.009801  ...       0.955466       0.828709   
2753           0.009415      -0.009203  ...       0.949628       0.826336   
2754           0.009867      -0.009762  ...       0.947967       0.836802   

Function                                                              \
Band     EEG Fpz-Cz SDR EEG Fpz-Cz BDR EEG Fpz-Cz ATR EEG Fpz-Cz STR   
epoch                                                                  
2750           0.673293       1.022449       0.868205       0.709102   
2751           0.671946       1.033851       0.858498       0.701913   
2752           0.685718       1.060795       0.867335       0.717679   
2753           0.668646       1.007109       0.870169       0.704113   
2754           0.649547       1.062096       0.882734       0.685200   

Function                                                              
Band     EEG Fpz-Cz BTR EEG Fpz-Cz SAR EEG Fpz-Cz BAR EEG Fpz-Cz BSR  
epoch                                                                 
2750           1.076828       0.816745       1.240292       1.518580  
2751           1.079958       0.817606       1.257962       1.538592  
2752           1.110239       0.827453       1.280057       1.546986  
2753           1.060530       0.809169       1.218764       1.506192  
2754           1.120393       0.776225       1.269231       1.635134  

[5 rows x 274 columns]

In [4]:
train_X.head()

Function Spect_slope                                                     \
Band      EEG Fpz-Cz EEG Fpz-Cz Delta EEG Fpz-Cz Theta EEG Fpz-Cz Alpha   
epoch                                                                     
2750      -24.945670       -23.984789        -0.226054         0.237275   
2751      -23.769731       -22.905892        -0.215343         0.237430   
2752      -24.182665       -23.310922        -0.224329         0.237022   
2753      -24.692606       -23.712681        -0.232493         0.227249   
2754      -24.218212       -23.309319        -0.238958         0.236427   

Function                                                                 \
Band     EEG Fpz-Cz Sigma EEG Fpz-Cz Beta EEG Fpz-Cz DOR EEG Fpz-Cz TOR   
epoch                                                                     
2750             0.173856        0.780128       0.961481       0.009062   
2751             0.172868        0.779275       0.963658       0.009060   
2752             0.175659        0.779543       0.963952       0.009276   
2753             0.168636        0.777726       0.960315       0.009415   
2754             0.174723        0.778467       0.962471       0.009867   

Function                                ...       AutoCorr                 \
Band     EEG Fpz-Cz AOR EEG Fpz-Cz SOR  ... EEG Fpz-Cz TDR EEG Fpz-Cz ADR   
epoch                                   ...                                 
2750          -0.009512      -0.006969  ...       0.949501       0.824361   
2751          -0.009989      -0.007273  ...       0.957307       0.821846   
2752          -0.009801      -0.007264  ...       0.955466       0.828709   
2753          -0.009203      -0.006829  ...       0.949628       0.826336   
2754          -0.009762      -0.007215  ...       0.947967       0.836802   

Function                                                              \
Band     EEG Fpz-Cz SDR EEG Fpz-Cz BDR EEG Fpz-Cz ATR EEG Fpz-Cz STR   
epoch                                                                  
2750           0.673293       1.022449       0.868205       0.709102   
2751           0.671946       1.033851       0.858498       0.701913   
2752           0.685718       1.060795       0.867335       0.717679   
2753           0.668646       1.007109       0.870169       0.704113   
2754           0.649547       1.062096       0.882734       0.685200   

Function                                                              
Band     EEG Fpz-Cz BTR EEG Fpz-Cz SAR EEG Fpz-Cz BAR EEG Fpz-Cz BSR  
epoch                                                                 
2750           1.076828       0.816745       1.240292       1.518580  
2751           1.079958       0.817606       1.257962       1.538592  
2752           1.110239       0.827453       1.280057       1.546986  
2753           1.060530       0.809169       1.218764       1.506192  
2754           1.120393       0.776225       1.269231       1.635134  

[5 rows x 273 columns]

# Deskewness

for lst in [train_X.columns[i:i+len(six_bands_n)] for i in range(0,len(train_X.columns),len(frbands_n))]:
    for col in lst:
        skw = stats.skew(train_X[[col]])
        x,lamb = stats.boxcox(train_X[[col]])
        train_X[[col]] = x
        print('Feature/Skewness : %s/%f'%(col,skw))

# Feature Selection & Model Fitting

In [5]:
def Encoder_and_Estimator(en='st', est='logistic'):

    en_ = {
        'st': StandardScaler(),
        'mm': MinMaxScaler(feature_range=(0, 1)),
        'lab': preprocessing.LabelEncoder()
    }

    est_ = {
        'logistic': LogisticRegression(),
        'LDA': LinearDiscriminantAnalysis(),
        'DT': tree.DecisionTreeClassifier(),
        'SVC': SVC(C=0.8,kernel='linear')
    }

    return en_[en], est_[est]

### Encoding And Model generating

In [13]:
# Label encoding for toget
trY = Encoder_and_Estimator(en='lab')[0].fit_transform(train_Y)
# Encoding
encoder, estimator = Encoder_and_Estimator(en='st', est='logistic')
# Encoding
sc = Encoder_and_Estimator(en='mm')[0]

trX = encoder.fit_transform(train_X)

In [7]:
trX.shape

(22242, 273)

In [8]:
df = pd.DataFrame(trX)
df.columns = train_X.columns
df.to_csv('trX.csv')

### Filter Methods selection

In [8]:
trX = VIF_ANOVA_feature_selection(X=trX,Y=trY,sc=sc,con=50)

100%|██████████████████████████████████████████████████████████████████████████████| 233/233 [1:58:22<00:00, 30.48s/it]


In [9]:
trX.shape

(22242, 40)

In [10]:
df = pd.DataFrame(trX)
df.to_csv('Collinarity correction_50.csv')

In [11]:
temp = add_constant(trX)
VIF_ = [variance_inflation_factor(temp, i) for i in range(temp.shape[1])][1:]
df = pd.DataFrame(VIF_)
df.to_csv('VIF_50.csv')

# Model fitting

### Fitting

In [12]:
# All of the features
cv = cross_val_score(estimator, trX, trY, cv=5).mean()

with open("REPORT.csv", "a+") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['OverallAcc',cv])

C:\Users\vita7\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vita7\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\vita7\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vita7\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\vita7\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default so

# Level_0 ACC
cv_ = []

for i in list(train_X.columns.get_level_values(0).unique()):
    trX = encoder.fit_transform(train_X.xs(i,level = 0,axis=1))
    cv_.append(cross_val_score(estimator, trX, trY, cv=5).mean())

with open("REPORT.csv", "a+") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([""]+list(train_X.columns.get_level_values(0).unique()))
    writer.writerow(['Level_0_Acc']+cv_)

# Level_1 ACC
cv_=[]

for i in list(train_X.columns.get_level_values(1).unique()):
    trX = encoder.fit_transform(train_X.xs(i,level = 1,axis=1))
    cv_.append(cross_val_score(estimator, trX, trY, cv=5).mean())

with open("REPORT.csv", "a+") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([""]+list(train_X.columns.get_level_values(1).unique()))
    writer.writerow(['Level_1_Acc']+cv_)

# Level_2 ACC
cv_=[]

for i in list(train_X.columns):
    trX = encoder.fit_transform(train_X[[i]])
    cv_.append(cross_val_score(estimator, trX, trY, cv=5).mean())

with open("REPORT.csv", "a+") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([""]+list(train_X.columns))
    writer.writerow(['Level_2_Acc']+cv_)